<a href="https://colab.research.google.com/github/kwonbuyeon/2021-1_basicCarAI/blob/master/pre_trained_model%EC%9D%98_%EC%84%B1%EB%8A%A5_%EB%B9%84%EA%B5%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**pre-trained model의 성능 비교(top1_accrncy로 정확도 확인)**
------
20203030_권부연 

무작위로 선정된 10개의 폴더내의 데이터 를 비교하여 모델의 성능에 대한 결과를 출력하는 코드입니다.

imageNet의 valiant set를 활용했으며 

본 코드에서는 데이터 셋을 드라이브에 올린 후 /content/gdrive/MyDrive/dataset/val 위치에 데이터 폴더를 위치 시켰습니다


In [5]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


본 코드에서 사용하기 위한 imageNet의 카테고리를 가져옵니다. 

In [2]:
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

--2021-04-20 13:19:02--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0s      

2021-04-20 13:19:02 (113 MB/s) - ‘imagenet_classes.txt’ saved [10472/10472]



아래 코드에서 사용할 pre-trained 모델을 torch에서 가져옵니다. 현재 코드는 AlexNet을 가져오게 설정되어 있으며 원하는 모델로 변경을 원하시면 아래 코드에서 모델명을 바꿔 주면됩니다.

In [3]:
from torchvision import models
import torch

model = torch.hub.load('pytorch/vision:v0.9.0', 'alexnet', pretrained=True)
model.eval()

Downloading: "https://github.com/pytorch/vision/archive/v0.9.0.zip" to /root/.cache/torch/hub/v0.9.0.zip
Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

폴더 리스트를 만드는 부분입니다. 폴더 리스트는 10개의 폴더를 선택할 때와 선택된 데이터의 라벨을 구분할때 사용됩니다.

In [6]:
from PIL import Image
import os
import glob

folder = os.listdir('/content/gdrive/MyDrive/dataset/val')
folder.sort()
folderLen = len(folder)

input할 이미지를 변형하는 부분입니다.

In [7]:
from torchvision import transforms

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
])


본 코드는 무작위로 선정된 10개의 폴더내의 데이터 셋을 비교합니다. 

아래는 무작위로 10개의 폴더를 선택하는 코드입니다.

In [8]:
import random
test = random.sample(folder,10)
print(test)

import copy

f=copy.deepcopy(folder)
f.sort()
labeling = {f[i]:categories[i] for i in range(len(f))}

['n01494475', 'n03874599', 'n03657121', 'n01739381', 'n02028035', 'n02051845', 'n07717410', 'n02486410', 'n01685808', 'n02437312']


데이터를 탐색하여 모델의 판단 결과를 구해 정확도를 확인하는 부분입니다.


In [11]:
from PIL import Image
import os
import glob


totalRight =0
for folderNum in range(10):
    total=0
    right =0
    for filePath in glob.glob('/content/gdrive/MyDrive/dataset/val/'+test[folderNum]+'/*.JPEG'):
        file =Image.open(filePath)
        input_tensor = preprocess(file)
        input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

        if torch.cuda.is_available():
            input_batch = input_batch.to('cuda')
            model.to('cuda')

        with torch.no_grad():
            try: 
                output = model(input_batch)
            except: 
                print("pass")
                continue

        probabilities = torch.nn.functional.softmax(output[0], dim=0)

        total+= 1

        top1_prob, top1_catid = torch.topk(probabilities, 1)
        if categories[top1_catid[0]] == labeling[test[folderNum]]:
            right += 1
            

    print("\n current Accuracy :" , 100 * right / total)
    print("continue :", (folderNum+1)*100/10)
    totalRight += (100 * right / total)
    print("total accuracy :",totalRight/(folderNum+1))
    

pass

 current Accuracy : 59.183673469387756
continue : 10.0
total accuracy : 59.183673469387756

 current Accuracy : 16.0
continue : 20.0
total accuracy : 37.59183673469388
pass
pass
pass

 current Accuracy : 21.27659574468085
continue : 30.0
total accuracy : 32.1534230713562

 current Accuracy : 34.0
continue : 40.0
total accuracy : 32.61506730351715

 current Accuracy : 78.0
continue : 50.0
total accuracy : 41.69205384281372

 current Accuracy : 40.0
continue : 60.0
total accuracy : 41.410044869011436

 current Accuracy : 24.0
continue : 70.0
total accuracy : 38.9228956020098

 current Accuracy : 54.0
continue : 80.0
total accuracy : 40.807533651758575

 current Accuracy : 52.0
continue : 90.0
total accuracy : 42.0511410237854

 current Accuracy : 58.0
continue : 100.0
total accuracy : 43.64602692140686


current Accuracy 현재 폴더 내의 데이터 셋의 정확도

continue 진행도

total accuracy : 현재까지 나온 정확도의 평균